# Suchfunktionen und Bücher löschen

## 🎯 Lernziele

Nach dieser Unterrichtseinheit können Sie:
- Suchfunktionen implementieren (nach Titel, Autor, ISBN)
- Mit Strings arbeiten (Teilstring-Suche, case-insensitive)
- Bücher aus der Library löschen
- Weitere Custom Exceptions erstellen
- Komplexere Tests mit mehreren Testfällen schreiben

---

## 1. Rückblick: Was haben wir bereits?

Lassen Sie uns unsere bisherigen Klassen zusammenfassen:

In [ ]:
# Book-Klasse (aus dem ersten Skript)
class Book:
    def __init__(self, title, isbn, author):
        self.title = title
        self.isbn = isbn
        self.author = author

# Custom Exception
class DuplicateISBNError(Exception):
    """Fehler: Ein Buch mit dieser ISBN existiert bereits"""
    pass

# Library-Klasse (aus dem zweiten Skript)
class Library:
    def __init__(self):
        """Konstruktor: Erstellt eine leere Bibliothek"""
        self._books = {}  # Dictionary: ISBN -> Book-Objekt
    
    def book_count(self):
        """Gibt die Anzahl der Bücher zurück"""
        return len(self._books)
    
    def add_book(self, book):
        """Fügt ein Buch zur Bibliothek hinzu"""
        if book.isbn in self._books:
            raise DuplicateISBNError(f"Buch mit ISBN {book.isbn} existiert bereits")
        self._books[book.isbn] = book
    
    def get_book(self, isbn):
        """Gibt ein Buch anhand der ISBN zurück"""
        return self._books.get(isbn)

print("✅ Bisherige Klassen geladen")

**Was kann unsere Library bereits?**
- ✅ Bücher hinzufügen (`add_book`)
- ✅ Bücher über ISBN abrufen (`get_book`)
- ✅ Anzahl der Bücher zählen (`book_count`)
- ✅ Duplikate verhindern

**Was fehlt noch?**
- ❌ Bücher suchen (nach Titel oder Autor)
- ❌ Bücher löschen
- ❌ Alle Bücher anzeigen

→ **Das implementieren wir heute!**

---

## 2. Warum brauchen wir eine Suchfunktion?

### Das Problem

Aktuell können wir nur Bücher über die **exakte ISBN** finden:

```python
book = library.get_book("9781234567890")  # Funktioniert
```

**Aber was, wenn ich...**
- ...die ISBN nicht kenne?
- ...nur den Titel oder Autor weiß?
- ...alle Bücher von einem bestimmten Autor finden will?

→ **Wir brauchen eine flexible Suchfunktion!**

### Anforderungen an die Suche

1. **Suche nach Titel:** "Python" findet "Python Testing" und "Learn Python"
2. **Suche nach Autor:** "Mustermann" findet alle Bücher von "Max Mustermann"
3. **Suche nach ISBN:** Wie bisher
4. **Teilstring-Suche:** "Pyth" findet "Python Testing"
5. **Case-insensitive:** "python" findet auch "Python" (Groß-/Kleinschreibung egal)

---

## 3. String-Operationen: Grundlagen

Bevor wir die Suchfunktion implementieren, wiederholen wir wichtige **String-Operationen**:

In [ ]:
# String-Grundlagen
text = "Python Testing"

# 1. Kleinbuchstaben (lowercase)
print("Original:", text)
print("Kleinbuchstaben:", text.lower())
print()

# 2. Teilstring-Prüfung (substring check)
print("Enthält 'Python'?", "Python" in text)
print("Enthält 'Java'?", "Java" in text)
print()

# 3. Case-insensitive Suche
suchbegriff = "python"  # Kleingeschrieben!
print(f"Enthält '{suchbegriff}' (case-insensitive)?")
print(suchbegriff.lower() in text.lower())
print()

# 4. Teilstring-Suche
print("Enthält 'Pyth'?", "Pyth" in text)
print("Enthält 'Test'?", "Test" in text)

**Wichtige String-Operationen:**

| Operation | Bedeutung | Beispiel |
|-----------|-----------|----------|
| `text.lower()` | Wandelt in Kleinbuchstaben um | `"Python".lower()` → `"python"` |
| `text.upper()` | Wandelt in Großbuchstaben um | `"python".upper()` → `"PYTHON"` |
| `substring in text` | Prüft, ob Teilstring enthalten ist | `"Pyth" in "Python"` → `True` |
| `text.startswith(s)` | Prüft, ob Text mit s beginnt | `"Python".startswith("Py")` → `True` |
| `text.endswith(s)` | Prüft, ob Text mit s endet | `"Python".endswith("on")` → `True` |

### Warum `.lower()` für die Suche?

**Problem ohne `.lower()`:**

In [ ]:
# Problem: Groß-/Kleinschreibung
titel = "Python Testing"
suche = "python"  # Kleingeschrieben

# Direkte Suche (funktioniert NICHT)
print(f"'{suche}' in '{titel}'?", suche in titel)  # False!

# Lösung: Beide in Kleinbuchstaben umwandeln
print(f"'{suche}' in '{titel}' (case-insensitive)?")
print(suche.lower() in titel.lower())  # True!

**Merke:** Für case-insensitive Suche immer **beide** Strings mit `.lower()` umwandeln!

---

## 4. Feature: Suche nach Titel

### Anforderung

Ich will alle Bücher finden, deren **Titel** einen bestimmten Suchbegriff enthalten.

**Beispiel:**
- Suche nach "Python" → findet "Python Testing", "Learn Python"
- Suche nach "Test" → findet "Python Testing", "Unit Testing"

### SCHRITT 1: Test schreiben (RED)

In [ ]:
# TEST 1: Suche nach Titel
def test_search_by_title():
    """Test: Suche nach Titel findet passende Bücher"""
    # Arrange
    library = Library()
    book1 = Book("Python Testing", "ISBN-001", "Max Mustermann")
    book2 = Book("Learn Python", "ISBN-002", "Anna Schmidt")
    book3 = Book("Java Basics", "ISBN-003", "Tom Müller")
    library.add_book(book1)
    library.add_book(book2)
    library.add_book(book3)
    
    # Act
    results = library.search(title="Python")
    
    # Assert
    assert len(results) == 2, f"Erwartet: 2 Bücher, Bekommen: {len(results)}"
    assert book1 in results, "book1 sollte gefunden werden"
    assert book2 in results, "book2 sollte gefunden werden"
    assert book3 not in results, "book3 sollte NICHT gefunden werden"
    print("✅ Test bestanden: Suche nach Titel")

# test_search_by_title()  # Wird fehlschlagen: search() existiert noch nicht

**Was testet dieser Test?**

1. Wir haben 3 Bücher: 2 mit "Python" im Titel, 1 ohne
2. Wir suchen nach "Python"
3. Wir erwarten **genau 2 Ergebnisse**
4. Die richtigen Bücher sollen gefunden werden

### SCHRITT 2: Implementation (GREEN)

In [ ]:
# Erweiterte Library-Klasse mit search()
class Library:
    def __init__(self):
        self._books = {}
    
    def book_count(self):
        return len(self._books)
    
    def add_book(self, book):
        if book.isbn in self._books:
            raise DuplicateISBNError(f"Buch mit ISBN {book.isbn} existiert bereits")
        self._books[book.isbn] = book
    
    def get_book(self, isbn):
        return self._books.get(isbn)
    
    def search(self, title=None, author=None, isbn=None):
        """Sucht Bücher nach Titel, Autor oder ISBN
        
        Args:
            title: Suchbegriff für Titel (optional)
            author: Suchbegriff für Autor (optional)
            isbn: Suchbegriff für ISBN (optional)
        
        Returns:
            Liste von Book-Objekten, die dem Suchkriterium entsprechen
        """
        results = []  # Leere Liste für Ergebnisse
        
        # Durchlaufe alle Bücher im Dictionary
        for book in self._books.values():
            # Suche nach Titel
            if title and title.lower() in book.title.lower():
                results.append(book)
        
        return results

# Test ausführen
test_search_by_title()

**Erklärung der `search()`-Methode:**

```python
def search(self, title=None, author=None, isbn=None):
```
- **Parameter mit Standardwert `None`:** Alle Parameter sind optional
- Man kann aufrufen: `search(title="Python")` oder `search(author="Max")`

```python
for book in self._books.values():
```
- `.values()` gibt alle **Werte** (Book-Objekte) aus dem Dictionary zurück
- Wir durchlaufen jedes Buch

```python
if title and title.lower() in book.title.lower():
```
- `if title`: Prüft, ob ein Titel-Suchbegriff angegeben wurde
- `title.lower() in book.title.lower()`: Case-insensitive Teilstring-Suche
- Wenn gefunden → Buch zur Ergebnisliste hinzufügen

### Test: Case-insensitive Suche

In [ ]:
# TEST 2: Case-insensitive Suche
def test_search_case_insensitive():
    """Test: Suche ignoriert Groß-/Kleinschreibung"""
    # Arrange
    library = Library()
    book = Book("Python Testing", "ISBN-001", "Max Mustermann")
    library.add_book(book)
    
    # Act & Assert
    # Alle diese Suchen sollten das Buch finden:
    assert len(library.search(title="python")) == 1, "Kleinschreibung sollte funktionieren"
    assert len(library.search(title="PYTHON")) == 1, "Großschreibung sollte funktionieren"
    assert len(library.search(title="PyThOn")) == 1, "Gemischt sollte funktionieren"
    print("✅ Test bestanden: Case-insensitive Suche")

# Test ausführen
test_search_case_insensitive()

---

## 5. Feature: Suche nach Autor

### Anforderung

Ich will alle Bücher finden, deren **Autor** einen bestimmten Suchbegriff enthält.

### SCHRITT 1: Test schreiben (RED)

In [ ]:
# TEST 3: Suche nach Autor
def test_search_by_author():
    """Test: Suche nach Autor findet passende Bücher"""
    # Arrange
    library = Library()
    book1 = Book("Buch A", "ISBN-001", "Max Mustermann")
    book2 = Book("Buch B", "ISBN-002", "Max Schmidt")
    book3 = Book("Buch C", "ISBN-003", "Anna Müller")
    library.add_book(book1)
    library.add_book(book2)
    library.add_book(book3)
    
    # Act
    results = library.search(author="Max")
    
    # Assert
    assert len(results) == 2, f"Erwartet: 2 Bücher, Bekommen: {len(results)}"
    assert book1 in results, "book1 sollte gefunden werden"
    assert book2 in results, "book2 sollte gefunden werden"
    assert book3 not in results, "book3 sollte NICHT gefunden werden"
    print("✅ Test bestanden: Suche nach Autor")

# test_search_by_author()  # Wird fehlschlagen: author-Suche noch nicht implementiert

### SCHRITT 2: Implementation (GREEN)

In [ ]:
# Erweiterte search()-Methode
class Library:
    def __init__(self):
        self._books = {}
    
    def book_count(self):
        return len(self._books)
    
    def add_book(self, book):
        if book.isbn in self._books:
            raise DuplicateISBNError(f"Buch mit ISBN {book.isbn} existiert bereits")
        self._books[book.isbn] = book
    
    def get_book(self, isbn):
        return self._books.get(isbn)
    
    def search(self, title=None, author=None, isbn=None):
        """Sucht Bücher nach Titel, Autor oder ISBN"""
        results = []
        
        for book in self._books.values():
            # Suche nach Titel
            if title and title.lower() in book.title.lower():
                results.append(book)
            # Suche nach Autor
            elif author and author.lower() in book.author.lower():
                results.append(book)
        
        return results

# Test ausführen
test_search_by_author()

**Was haben wir hinzugefügt?**

```python
elif author and author.lower() in book.author.lower():
    results.append(book)
```

- `elif`: "Sonst, wenn" (nur prüfen, wenn Titel-Suche nicht zutraf)
- Gleiche Logik wie bei Titel-Suche
- Case-insensitive Teilstring-Suche im Autor-Namen

---

## 6. Feature: Suche nach ISBN

### Anforderung

Ich will auch über die `search()`-Methode nach ISBN suchen können (zusätzlich zu `get_book()`).

### Test und Implementation

In [ ]:
# TEST 4: Suche nach ISBN
def test_search_by_isbn():
    """Test: Suche nach ISBN findet das Buch"""
    # Arrange
    library = Library()
    book = Book("Python Testing", "ISBN-001", "Max Mustermann")
    library.add_book(book)
    
    # Act
    results = library.search(isbn="ISBN-001")
    
    # Assert
    assert len(results) == 1, "Genau 1 Buch sollte gefunden werden"
    assert results[0] == book, "Das richtige Buch sollte gefunden werden"
    print("✅ Test bestanden: Suche nach ISBN")

# Erweiterte search()-Methode
class Library:
    def __init__(self):
        self._books = {}
    
    def book_count(self):
        return len(self._books)
    
    def add_book(self, book):
        if book.isbn in self._books:
            raise DuplicateISBNError(f"Buch mit ISBN {book.isbn} existiert bereits")
        self._books[book.isbn] = book
    
    def get_book(self, isbn):
        return self._books.get(isbn)
    
    def search(self, title=None, author=None, isbn=None):
        """Sucht Bücher nach Titel, Autor oder ISBN"""
        results = []
        
        for book in self._books.values():
            # Suche nach Titel
            if title and title.lower() in book.title.lower():
                results.append(book)
            # Suche nach Autor
            elif author and author.lower() in book.author.lower():
                results.append(book)
            # Suche nach ISBN
            elif isbn and isbn == book.isbn:
                results.append(book)
        
        return results

# Test ausführen
test_search_by_isbn()

**Hinweis:** Bei ISBN verwenden wir **exakte Suche** (`isbn == book.isbn`), nicht Teilstring-Suche!

### Test: Keine Ergebnisse

In [ ]:
# TEST 5: Suche ohne Ergebnisse
def test_search_no_results():
    """Test: Suche ohne Treffer gibt leere Liste zurück"""
    # Arrange
    library = Library()
    book = Book("Python Testing", "ISBN-001", "Max Mustermann")
    library.add_book(book)
    
    # Act
    results = library.search(title="Java")  # Nicht vorhanden
    
    # Assert
    assert len(results) == 0, "Keine Bücher sollten gefunden werden"
    assert results == [], "Ergebnis sollte leere Liste sein"
    print("✅ Test bestanden: Keine Ergebnisse")

# Test ausführen
test_search_no_results()

---

## 7. Feature: Buch löschen

### Anforderung

Ich will ein Buch aus der Library entfernen können.

**Was soll passieren?**
- Buch wird aus dem Dictionary gelöscht
- `book_count()` wird um 1 kleiner
- Wenn ISBN nicht existiert → Fehler werfen

### Neue Exception: BookNotFoundError

In [ ]:
# Neue Custom Exception
class BookNotFoundError(Exception):
    """Fehler: Buch wurde nicht gefunden"""
    pass

print("✅ BookNotFoundError definiert")

### SCHRITT 1: Test schreiben (RED)

In [ ]:
# TEST 6: Buch löschen
def test_remove_book():
    """Test: Buch kann gelöscht werden"""
    # Arrange
    library = Library()
    book = Book("Python Testing", "ISBN-001", "Max Mustermann")
    library.add_book(book)
    assert library.book_count() == 1, "Library sollte 1 Buch haben"
    
    # Act
    library.remove_book("ISBN-001")
    
    # Assert
    assert library.book_count() == 0, "Library sollte jetzt leer sein"
    assert library.get_book("ISBN-001") is None, "Buch sollte nicht mehr abrufbar sein"
    print("✅ Test bestanden: Buch löschen")

# test_remove_book()  # Wird fehlschlagen: remove_book() existiert noch nicht

### SCHRITT 2: Implementation (GREEN)

In [ ]:
# Finale Library-Klasse mit remove_book()
class Library:
    def __init__(self):
        self._books = {}
    
    def book_count(self):
        return len(self._books)
    
    def add_book(self, book):
        if book.isbn in self._books:
            raise DuplicateISBNError(f"Buch mit ISBN {book.isbn} existiert bereits")
        self._books[book.isbn] = book
    
    def get_book(self, isbn):
        return self._books.get(isbn)
    
    def search(self, title=None, author=None, isbn=None):
        """Sucht Bücher nach Titel, Autor oder ISBN"""
        results = []
        for book in self._books.values():
            if title and title.lower() in book.title.lower():
                results.append(book)
            elif author and author.lower() in book.author.lower():
                results.append(book)
            elif isbn and isbn == book.isbn:
                results.append(book)
        return results
    
    def remove_book(self, isbn):
        """Entfernt ein Buch aus der Bibliothek
        
        Args:
            isbn: ISBN des zu löschenden Buches
        
        Raises:
            BookNotFoundError: Wenn Buch nicht gefunden wurde
        """
        # Prüfen, ob Buch existiert
        if isbn not in self._books:
            raise BookNotFoundError(f"Buch mit ISBN {isbn} nicht gefunden")
        
        # Buch löschen
        del self._books[isbn]

# Test ausführen
test_remove_book()

**Erklärung:**

```python
if isbn not in self._books:
    raise BookNotFoundError(...)
```
- Prüft, ob ISBN **nicht** im Dictionary ist
- Wenn nicht vorhanden → Fehler werfen

```python
del self._books[isbn]
```
- `del` löscht einen Eintrag aus dem Dictionary
- Danach ist die ISBN nicht mehr vorhanden

### Test: Nicht existierendes Buch löschen

In [ ]:
# TEST 7: Nicht existierendes Buch löschen
def test_remove_nonexistent_book():
    """Test: Löschen eines nicht existierenden Buches wirft Fehler"""
    # Arrange
    library = Library()
    
    # Act & Assert
    try:
        library.remove_book("ISBN-NICHT-VORHANDEN")
        assert False, "BookNotFoundError wurde nicht geworfen!"
    except BookNotFoundError as e:
        print(f"✅ Test bestanden: Fehler geworfen: {e}")

# Test ausführen
test_remove_nonexistent_book()

---

## 8. Alle Tests zusammen ausführen

In [ ]:
# Alle Tests ausführen
print("=== Test-Suite für Library (erweitert) ===")
print()

print("Test 1: Suche nach Titel")
test_search_by_title()
print()

print("Test 2: Case-insensitive Suche")
test_search_case_insensitive()
print()

print("Test 3: Suche nach Autor")
test_search_by_author()
print()

print("Test 4: Suche nach ISBN")
test_search_by_isbn()
print()

print("Test 5: Suche ohne Ergebnisse")
test_search_no_results()
print()

print("Test 6: Buch löschen")
test_remove_book()
print()

print("Test 7: Nicht existierendes Buch löschen")
test_remove_nonexistent_book()
print()

print("=== Alle Tests bestanden! ✅ ===")

---

## 9. Praktisches Beispiel: Library in Aktion

In [ ]:
# Erstelle eine Bibliothek mit mehreren Büchern
library = Library()

# Bücher hinzufügen
library.add_book(Book("Python Testing", "ISBN-001", "Max Mustermann"))
library.add_book(Book("Learn Python", "ISBN-002", "Anna Schmidt"))
library.add_book(Book("Java Basics", "ISBN-003", "Tom Müller"))
library.add_book(Book("Advanced Python", "ISBN-004", "Max Mustermann"))
library.add_book(Book("Clean Code", "ISBN-005", "Robert Martin"))

print(f"Bibliothek hat {library.book_count()} Bücher")
print()

# Suche nach Titel
print("=== Suche nach 'Python' ===")
results = library.search(title="Python")
for book in results:
    print(f"  - {book.title} von {book.author}")
print()

# Suche nach Autor
print("=== Suche nach Autor 'Max' ===")
results = library.search(author="Max")
for book in results:
    print(f"  - {book.title} von {book.author}")
print()

# Buch löschen
print("=== Lösche 'Java Basics' ===")
library.remove_book("ISBN-003")
print(f"Bibliothek hat jetzt {library.book_count()} Bücher")
print()

# Suche nach gelöschtem Buch
print("=== Suche nach 'Java' ===")
results = library.search(title="Java")
if len(results) == 0:
    print("  Keine Bücher gefunden (wurde gelöscht!)")

---

## 10. Zusammenfassung

### Was haben wir gelernt?

#### 1. String-Operationen
- `.lower()` für case-insensitive Vergleiche
- `substring in text` für Teilstring-Suche
- Kombination: `search.lower() in text.lower()`

#### 2. Suchfunktion
- Flexible Suche nach Titel, Autor oder ISBN
- Optionale Parameter mit Standardwert `None`
- Durchlaufen aller Bücher mit `for book in self._books.values()`

#### 3. Löschen von Einträgen
- `del dictionary[key]` löscht einen Eintrag
- Prüfung mit `if key not in dictionary`
- Exception werfen bei nicht existierendem Eintrag

#### 4. Weitere Custom Exception
- `BookNotFoundError` für nicht gefundene Bücher

### Unsere Library kann jetzt:
- ✅ Bücher hinzufügen
- ✅ Bücher abrufen (über ISBN)
- ✅ Bücher suchen (nach Titel, Autor, ISBN)
- ✅ Bücher löschen
- ✅ Anzahl der Bücher zählen
- ✅ Duplikate verhindern

---

## 11. Praktische Übungen

### Übung 1: Mehrere Suchkriterien

Erstellen Sie eine Library mit 5 Büchern. Suchen Sie dann:
1. Alle Bücher mit "Python" im Titel
2. Alle Bücher von einem bestimmten Autor
3. Ein Buch über seine ISBN

In [ ]:
# Ihre Lösung hier:


### Übung 2: Löschen und Prüfen

Erstellen Sie eine Library mit 3 Büchern. Löschen Sie ein Buch und prüfen Sie dann:
1. Die Anzahl der Bücher hat sich verringert
2. Das gelöschte Buch ist nicht mehr auffindbar
3. Die anderen Bücher sind noch vorhanden

In [ ]:
# Ihre Lösung hier:


### Übung 3: Fehlerbehandlung

Schreiben Sie Code, der versucht:
1. Ein nicht existierendes Buch zu löschen
2. Ein Duplikat hinzuzufügen

Fangen Sie beide Fehler ab und geben Sie passende Meldungen aus.

In [ ]:
# Ihre Lösung hier:


### Übung 4: Test schreiben (Fortgeschritten)

Schreiben Sie einen Test, der überprüft, dass die Suche mehrere Bücher findet, wenn mehrere dem Suchkriterium entsprechen.

In [ ]:
def test_search_finds_multiple_books():
    """Test: Suche findet mehrere passende Bücher"""
    # Ihre Lösung hier:
    pass

# test_search_finds_multiple_books()

---

## 12. Ausblick: Was kommt als Nächstes?

### Heute haben wir:
- ✅ Suchfunktionen implementiert
- ✅ Bücher löschen können
- ✅ String-Operationen gelernt

### Als Nächstes:
- **pytest einführen:** Professionelles Testing-Framework
- **Fixtures:** Test-Daten wiederverwendbar machen
- **Parametrisierte Tests:** Viele Testfälle mit wenig Code

### Später kommen:
- **User-Klasse:** Benutzer verwalten
- **Ausleihe-System:** Bücher ausleihen und zurückgeben
- **Persistenz:** Daten in Dateien speichern

**Prinzip:** Schritt für Schritt, immer mit Tests! 🚀

---

## 🎓 Wichtige Begriffe

| Begriff | Bedeutung | Beispiel |
|---------|-----------|----------|
| **Teilstring** | Teil eines Strings | `"Pyth"` ist Teilstring von `"Python"` |
| **Case-insensitive** | Groß-/Kleinschreibung egal | `"python" == "PYTHON"` |
| **.lower()** | Wandelt in Kleinbuchstaben um | `"Python".lower()` → `"python"` |
| **in** | Prüft, ob enthalten | `"Py" in "Python"` → `True` |
| **del** | Löscht einen Dictionary-Eintrag | `del dict[key]` |
| **.values()** | Gibt alle Werte eines Dicts zurück | `for book in books.values()` |
| **Optionale Parameter** | Parameter mit Standardwert | `def search(title=None)` |

---

## 🎉 Herzlichen Glückwunsch!

Sie haben die Library-Klasse um wichtige Funktionen erweitert!

**Was Sie jetzt können:**
- ✅ Flexible Suchfunktionen implementieren
- ✅ Mit Strings arbeiten (case-insensitive, Teilstrings)
- ✅ Einträge aus Dictionaries löschen
- ✅ Mehrere Custom Exceptions verwenden
- ✅ Komplexe Tests schreiben

**Nächste Schritte:**
- Lösen Sie die Übungen
- Experimentieren Sie mit der erweiterten Library
- Im nächsten Skript: pytest und professionelles Testing!